In [3]:
import pandas as pd
import altair as alt
import numpy as np
from pathlib import Path
from helpers import read

import bottleneck as bn

# import re
# from algo import KLD_distance_overused
# import string
# from glob import glob
# from collections import Counter
# from datetime import datetime
from LPA import Corpus, sockpuppet_distance
from math import floor
from scipy.spatial.distance import cdist

from visualize import sockpuppet_matrix

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [74]:
freq = pd.read_csv(f"data/us_elections/np_freq/1.csv")
x = pd.merge(
    freq,
    freq.drop(columns=["party_simplified"]).groupby("document").sum(numeric_only=True),
    left_on="document",
    right_index=True,
)
x["frequency_in_document"] = x["frequency_in_document_x"] / x["frequency_in_document_y"]
x = x.drop(
    columns=["frequency_in_document_x", "frequency_in_document_y", "party_simplified"]
)
cc = x.pivot_table(
    index="document", columns="element", values="frequency_in_document"
).to_numpy()
sockpuppet_distance(cc, cc)
# df = pd.DataFrame(
#         cdist(cc[0], cc[1], metric="cityblock"),
#         index=corpus1.document_cat.categories,
#         columns=corpus2.document_cat.categories,
#     )
#     df /= df.max().max()
#     if res == "table":
#         c1n = getattr(corpus1, "name", "Corpus 2")
#         c2n = getattr(corpus2, "name", "Corpus 1")
#         if c1n == c2n:
#             c2n = c1n + " "
#         df = (
#             df.rename_axis(index=c1n)
#             .melt(ignore_index=False, var_name=c2n)
#             .reset_index()
#         )
#     return df

AttributeError: 'numpy.ndarray' object has no attribute 'distance_matrix'

In [5]:
df = pd.read_csv("data/us_elections/1976-2020-president.csv")[
    ["year", "candidatevotes", "party_simplified", "state"]
]
df = (
    df[df["party_simplified"].isin(["DEMOCRAT", "REPUBLICAN"])]
    .rename(
        columns={
            "state": "element",
            "year": "document",
            "candidatevotes": "frequency_in_document",
        }
    )
    .reset_index(drop=True)
)
df[df["party_simplified"] == "DEMOCRAT"].reset_index(drop=True).to_csv(
    "data/us_elections/np_freq/0.csv", index=False
)
# cs = []
# for party in ["DEMOCRAT", "DEMOCRAT"]:
#     corpus = Corpus(df[df["party_simplified"] == party].reset_index(drop=True))
#     dvr = corpus.create_dvr()
#     sigs = corpus.create_signatures(epsilon=1/(len(dvr) * 2))
#     cs.append(corpus)
# spd = sockpuppet_distance(*cs)
# sockpuppet_matrix(spd, "DEMOCRAT", "DEMOCRAT 1") #.save("dem_vs_dem.html")

In [34]:
num_to_year = {
    14: 1996,
    15: 1999,
    16: 2003,
    17: 2006,
    18: 2009,
    19: 2013,
    20: 2015,
    21: 2018,
    22: 2019,
    23: 2020,
    24: 2021,
    25: 2022,
}

### Preprocessing

In [35]:
# heb_cols = ["שם ישוב", "סמל ישוב", "בזב", "מצביעים", "פסולים", "כשרים"]
# eng_cols = ["town_name", "town_code", "bzb", "voters", "valid_votes", "invalid_votes"]
# years = ("2015", "2019a", "2019b", "2020", "2021") #  ("2015 - hebrew", "2019 - hebrew",
# drop_cols = {
#     "2019b": ["סמל ועדה"],
#     "2020": ["סמל ועדה", "Unnamed: 37"],
#     "2021": ["סמל ועדה", "Unnamed: 46"],
# } | {y: [] for y in years[:-3]}
full_df = []
# kv_df = []
for num, year in num_to_year.items():
    if num <= 20:
        df = pd.read_excel(f"data/elections/results_{num}.xls")
    else:
        enc = {"encoding": "iso8859_8"} if num != 25 else {}
        df = pd.read_csv(f"data/elections/results_{num}.csv", **enc)
    print(year)
    df.loc[
        ((df["שם ישוב"] == "מעטפות חיצוניות") | (df["שם ישוב"] == "מעטפות כפולות")),
        "סמל ישוב",
    ] = 99999
    df = df.drop(
        columns=set(df.columns.tolist())
        & {
            "קלפי",
            "פיצול",
            "שם ישוב",
            "כתובת",
            "בעלי זכות בחירה",
            "בזב",
            "נפה",
            "מצביעים",
            "כשרים",
            "פסולים",
            "מספר קלפי",
            "ברזל",
            "סמל ועדה",
            "שופט",
            "ריכוז",
            "סמל קלפי",
        }
    )
    df = df.rename(columns={"סמל ישוב": "element"})
    full_df.append(
        df.groupby("element", as_index=False)
        .sum()
        .melt(id_vars=["element"], value_name="frequency_in_document")
        .assign(document=year)
    )
#     kv_df.append(df[["town_code", "town_name"]])
#     emet_df.append(df[df["party"] == "אמת"][["document","town_code", "votes"]].rename(columns={"town_code": "element", "votes": "frequency_in_document"}).reset_index(drop=True))

full_df = pd.concat(full_df)
full_df = (
    full_df[full_df["frequency_in_document"] > 0]
    .reset_index(drop=True)
    .replace({"אמת ": "אמת"})
)
for variable in full_df["variable"].drop_duplicates().to_list():
    full_df[full_df["variable"] == variable].drop(columns=["variable"]).to_csv(
        f"data/elections/parties/{variable}.csv", index=False
    )
# emet_df["element"] = emet_df["element"].replace(9999, 99999)
# emet_df
full_df = full_df.rename(columns={"element": "city", "variable": "element"})

1996
1999
2003
2006
2009
2013
2015
2018
2019
2020
2021
2022


In [38]:
cs = []
for i in [3000, 3000]:
    cs.append(
        full_df[full_df["city"] == i].reset_index(drop=True).drop(columns=["city"])
    )

In [5]:
k = ["שם יישוב", "סמל יישוב"]
v = ["name", "element"]
kv = pd.read_excel("data/elections/bycode2021.xlsx")[k].rename(columns=dict(zip(k, v)))
kv = pd.concat(
    [kv, pd.DataFrame(["מעטפות חיצוניות", 99999], index=v).T], ignore_index=True
)

# pd.DataFrame.from_dict(dict(zip(v, )), orient="columns")

In [40]:
css = []
for f in cs:
    corpus = Corpus(f)
    dvr = corpus.create_dvr()
    corpus.create_signatures(epsilon=1 / (len(dvr) * 2))
    css.append(corpus)

# print(css)
spd = sockpuppet_distance(*css)
sockpuppet_matrix(spd, "Tel Aviv", "Tel Aviv 2")

/home/alexzabbey/.virtualenvs/lpa/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### LPA

In [6]:
corpus = Corpus(freq=emet_df)
dvr = corpus.create_dvr()
pd.merge(dvr, kv).head(20)

,element,global_weight,name
0,3000,0.105517,ירושלים
1,6100,0.053490,בני ברק
2,99999,0.047087,מעטפות חיצוניות
3,70,0.045611,אשדוד
4,5000,0.043848,תל אביב -יפו
5,7400,0.032748,נתניה
6,6600,0.030517,חולון
7,9000,0.029700,באר שבע
8,7900,0.028214,פתח תקווה
9,8300,0.021468,ראשון לציון


In [7]:
epsilon_frac = 2
epsilon = 1 / (len(dvr) * epsilon_frac)
epsilon

0.00039184952978056425

In [8]:
prevelent = floor(len(corpus) * 0.75)
print(prevelent)
signatures, most_significant, temporary_array = corpus.create_signatures(
    epsilon=epsilon, most_significant=30, sig_length=500, prevelent=prevelent
)
display(signatures[0].head(10))

9


99999   -0.322609
1309    -0.087919
294      0.071905
6600    -0.071485
289      0.070993
633      0.061211
5000    -0.058087
139      0.055845
167      0.052052
7400    -0.051288
Name: 1996, dtype: float64